In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly 
import plotly.graph_objects as go
import plotly.express as px

from sklearn import linear_model
%matplotlib qt
pd.set_option('mode.chained_assignment', None)

In [ ]:
class Data:

    data: pd.DataFrame = None

    def __init__(self, data=None,name='python data'):
        self.data = data
        self.name= name
        self.desc=f'loaded from python'

    def load(self, path=''):
        while (path == '') or (not os.path.isfile(path)):
            path = input(f"Input data path: ")
        _path_list = (path).split('\\')
        self.folder = '\\'.join(_path_list[:-1])
        self.name = _path_list[-1]
        print(f'Loaded: {self.name}')
        self._raw_data = pd.read_csv(path)
        self.desc=f'loaded from: {path}'
        return self

    def build_data(self,
                   time_in_minutes=False,
                   temperature=(-np.inf, np.inf),
                   viscosity=(0, np.inf),
                   time=(0, np.inf)):
        self.conditions = dict(temperature=temperature,
                               viscosity=viscosity,
                               time=time)

        temperature_cond = ((temperature[0] < self._raw_data['Temperature']) &
                            (self._raw_data['Temperature'] < temperature[1]))
        viscosity_cond = ((viscosity[0] < self._raw_data['Viscosity']) &
                          (self._raw_data['Viscosity'] < viscosity[1]))
        time_cond = ((time[0] < self._raw_data['time']) &
                     (self._raw_data['time'] < time[1]))

        self.data = self._raw_data[temperature_cond & viscosity_cond
                                   & time_cond]
        if time_in_minutes: self.data['time'] / 60
        return self.data

    def temp_plot(self,
                  name,
                  y2_axix=False,
                  symbol='circle',
                  colors=plotly.colors.qualitative.Plotly):
        verboses = self.data[f'{name}_verbose'].unique()
        rules = zip(
            verboses,
            [round(i) for i in np.linspace(0, len(colors) - 1, len(verboses))],
        )
        colors_rule = {key: colors[color] for key, color in rules}

        scatter = go.Scatter(
            x=self.data['time'],
            y=self.data[name],
            name=name,
            mode='markers',
            yaxis='y2' if y2_axix else 'y',
            text=self.data[f'{name}_verbose'],
            hovertemplate='%{y}<br>%{text}',
            marker=dict(
                size=5,
                symbol=symbol,
                color=self.data[f'{name}_verbose'].replace(colors_rule),
            ),
        )
        return scatter

    @property
    def groupT(self):
        return self.data[['Temperature', 'Viscosity']].groupby(by='Temperature')

    @property
    def groupV(self):
        return self.data[['Temperature', 'Viscosity']].groupby(by='Viscosity')


# Load raw and save ready data

In [ ]:
# path = f'Experiments\MultiplyTemperature\Exp5(12.5)\Exp5_down.csv'

path = ''
while (path == '') or (not os.path.isfile(path)):
    path = input(f"Input data path: ")
_path_list = (path).split('\\')
folder_path = '\\'.join(_path_list[:-1])
data_name = _path_list[-1]
print(f'    Folder:\n{folder_path}\n    Data name:\n {data_name}')

In [ ]:
df1 = pd.read_csv('Experiments\\MultiplyTemperature\\Exp0(0)\\Exp0_up1.csv',
                  index_col=0)
# df2 = pd.read_csv('Experiments\\MultiplyTemperature\\Exp0(0)\\Exp0_up2.csv',index_col=0)
# df3 = pd.read_csv('Experiments\\MultiplyTemperature\\Exp0(0)\\Exp0_up3.csv',index_col=0)
# df4 = pd.read_csv('Experiments\\MultiplyTemperature\\Exp0(0)\\Exp0_up4.csv',index_col=0)


In [ ]:
df1

In [ ]:
rules = {
    1: 'image_sweep_check',
    2: 'combine_check',  # 'inner_processor_check':'OK_inner'
    0: 'OK'
}
col_rules = {
    'Viscosity_mark': 'Viscosity_verbose',
    'Temperature_mark': 'Temperature_verbose',
}

df1 = df1.rename(columns=col_rules)
df1[['Viscosity_verbose','Temperature_verbose']] =\
    df1[['Viscosity_verbose','Temperature_verbose']].replace(rules)

# df2 = df2.rename(columns=col_rules)
# df2[['Viscosity_verbose','Temperature_verbose']] = df2[['Viscosity_verbose','Temperature_verbose']].replace(rules)
# df2['time'] =df2['time']+ df1['time'].max()+1

# df3 = df3.rename(columns=col_rules)
# df3[['Viscosity_verbose','Temperature_verbose']] = df3[['Viscosity_verbose','Temperature_verbose']].replace({np.nan:'OK'})
# df3['time'] =df3['time']+ df2['time'].max()+1

# df4 = df4.rename(columns=col_rules)
# df4[['Viscosity_verbose','Temperature_verbose']] = df4[['Viscosity_verbose','Temperature_verbose']].replace({np.nan:'OK'})
# df4['time'] =df4['time']+ df3['time'].max()+1
# df = pd.concat([df1,df2,df3,df4])

df = df1.dropna(subset=['Viscosity', 'Temperature'])
df

In [ ]:
np.any(df['time'].duplicated())

In [ ]:
assert not os.path.isfile(f'{folder_path}\AData_{data_name}'), 'Data already exists'
df.to_csv(f'{folder_path}\AData_{data_name}', index=0)

# Load

In [ ]:
base = Data()
base.load('Experiments\MultiplyTemperature\Exp0(0)\AData_Exp0_up.csv')
base.build_data(
    # time=(172, np.inf),
    viscosity=(40, 350),
    temperature=(13.8,41)
)


# Time plots

In [ ]:
df =base.data
# df = test_data

fig, ax_v = plt.subplots()
ax_T = ax_v.twinx()

ax_v.scatter(df['time'], df['Viscosity'], color='red', marker='.')
ax_v.set_ylabel('Viscosity', color='red')

ax_T.scatter(df['time'], df['Temperature'], color='blue', marker='.')
ax_T.set_ylabel('Temperature', color='blue')


In [ ]:
base_plot=base
fig = go.Figure()
fig.add_traces([
    base_plot.temp_plot('Viscosity', colors=plotly.colors.sequential.Aggrnyl),
    base_plot.temp_plot('Temperature',
                        y2_axix=True,
                        symbol='star',
                        colors=plotly.colors.sequential.Sunsetdark_r)
])
fig.update_layout(
    hovermode="x unified",
    margin=dict(l=0, r=0, b=0, t=0, pad=0),
    yaxis2=dict(
        overlaying='y',
        side='right',  # range=[15, 20],
    ),  # xaxis=dict(range=[0, 500], ),
)
fig.show()

# Filtration

In [ ]:
def z_filter(data: pd.Series):
    mean = data.mean()
    s = data.std(ddof=0)+1e-50
    z_score= np.abs((data-mean) / s) <1.5
    return z_score

def iqr_filter(data: pd.Series):
    whisker_width=0.5
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    return (data >= q1 - whisker_width*iqr) & (data <= q3 + whisker_width*iqr)

def iqr2_filter(data: pd.Series):
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    
    return np.abs((data - data.median()) / iqr) < 2

In [ ]:
filter_func = iqr2_filter
T_group = base.groupT['Viscosity']

test = Data(base.data[T_group.apply(filter_func).droplevel([0]).reset_index(drop=True).to_numpy()],
                 name=f'filtered {base.name}')

# Temterature plots

In [ ]:
base_plot = base

fig = go.Figure()

fig = px.scatter(
    base_plot.data,
    x="Temperature",
    y="Viscosity",
    color='Viscosity_verbose',
)

## Mean and median
fig.add_traces([
    go.Scatter(name='Mean',
               x=base_plot.groupT.mean().index,
               y=base_plot.groupT.mean()['Viscosity'],
               mode='lines',
               line=dict(color='red')),
    go.Scatter(name='Median',
               x=base_plot.groupT.median().index,
               y=base_plot.groupT.median()['Viscosity'],
               mode='lines',
               line=dict(color='black')),
])

## Comparation
# test_plot= test
# fig.add_trace(
#     go.Scatter(name='Test',
#                x=test_plot.data['Temperature'],
#                y=test_plot.data['Viscosity'],
#                mode='markers',
#                line=dict(color='green'),
#                marker=dict(size=10)), )

# fig.add_traces([
#     go.Scatter(name='Mean comp',
#                x=test_plot.groupT.mean().index,
#                y=test_plot.groupT.mean()['Viscosity'],
#                mode='lines',
#                line=dict(color='red', width=2, dash='dot')),
#     go.Scatter(name='Median comp',
#                 x=test_plot.groupT.median().index,
#                 y=test_plot.groupT.median()['Viscosity'],
#                 mode='lines',
#                 line=dict(color='black', width=2, dash='dot')),
# ])

##Regression
x = reg_data.index
fig.add_trace(
    go.Scatter(
        name='Regression',
        x=x,
        y=reg_func(x),
        mode='lines',
        line=dict(color='green', width=2, dash='dot'),
    ), )

fig.update_layout(hovermode="x unified", margin=dict(l=0, r=0, b=0, t=0, pad=0))
fig.show()

# Regression

In [ ]:
def regress(data):
    T = np.array(data.index)
    nu = np.array(data['Viscosity'])

    x_T = 1 / (T+273.15)

    X = np.array([x_T]).T
    Y = np.log(nu)

    reg = linear_model.LinearRegression(fit_intercept=True)
    reg.fit(X, Y)
    w = reg.coef_[0]
    w0 = reg.intercept_

    nu0 = np.exp(w0)
    EkJ = 8.314 * w / 1000

    def approximation(T,*, nu0=nu0, EkJ=EkJ):
        TT = 1 / (T+273.15)
        nu = nu0 * np.exp(EkJ * 1000 / 8.314 * TT)
        return nu

    parametrs = dict(nu0=nu0, EkJ=EkJ)

    return parametrs, approximation

In [ ]:
reg_data = base.groupT.median()
reg_name= base.name

params,reg_func= regress(reg_data)
print(
    f"Experiment= {reg_name}",
    f"nu0=  {params['nu0']: 6.3e}",
    f"E  =  {params['EkJ']: 6.3f} kJ",
    sep='\n',
)
reg_func(100)

In [ ]:
reg_func(100,nu0= 1)

## Saving regression

In [ ]:
dfp = dfs_mean

In [ ]:
assert not os.path.isfile(f'{folder_path}\BReg_{data_name[6:]}'), 'Data already exists'
dfp.to_csv(f'{folder_path}\BReg_{data_name[6:]}', index=0)